In [ ]:
#pip install fastapi uvicorn joblib tensorflow scikit-learn yfinance
from fastapi import FastAPI
import yfinance as yf
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import joblib

app = FastAPI()

# Carrega o modelo e o scaler
modelo = load_model('modelo_45dias.h5')
scaler = joblib.load('scaler_45dias.pkl')

@app.get("/")
def home():
    return {"mensagem": "API de previsão de fechamento AAPL com janela de 45 dias."}

@app.get("/prever")
def prever(data_final: str = "2025-05-20"):
    # Baixar os dados
    df = yf.download("AAPL", start="2018-01-01", end=data_final)
    df.columns = df.columns.droplevel(0)
    df.columns.name = None
    df.columns = ['Preço_Fechamento', 'Máxima', 'Mínima', 'Preço_Abertura', 'Volume_Negociado']
    df.reset_index(inplace=True)

    # Feature: retorno suavizado
    df['Retorno_Diário'] = df['Preço_Fechamento'].pct_change()
    df['Retorno_MM3'] = df['Retorno_Diário'].rolling(window=3).mean()
    df.dropna(inplace=True)

    # Selecionar últimas 45 linhas e transformar
    variaveis = ['Preço_Fechamento', 'Máxima', 'Mínima', 'Volume_Negociado', 'Retorno_MM3']
    ultimos_dados = df[variaveis].iloc[-45:]
    X_input = scaler.transform(ultimos_dados)
    X_input = X_input.reshape(1, 45, len(variaveis))

    # Prever
    previsao_normalizada = modelo.predict(X_input)
    previsao = scaler.inverse_transform(np.hstack([previsao_normalizada, np.zeros((1, len(variaveis)-1))]))[0, 0]

    return {"previsao_proximo_fechamento": round(previsao, 2)}


In [ ]:
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
